In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_CB")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [18]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
import numpy as np


def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [22]:
# from transformers import set_seed
#
# set_seed(42)


In [23]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [24]:
loha_model = get_peft_model(model, peft_config)
loha_model.print_trainable_parameters()

trainable params: 1,181,955 || all params: 110,666,502 || trainable%: 1.0680


In [34]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_cb"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=loha_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [36]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'train_runtime': 38.8257, 'train_samples_per_second': 10.302, 'train_steps_per_second': 10.302, 'train_loss': 1.3028948974609376, 'epoch': 3.64}


TrainOutput(global_step=400, training_loss=1.3028948974609376, metrics={'train_runtime': 38.8257, 'train_samples_per_second': 10.302, 'train_steps_per_second': 10.302, 'total_flos': 24308644900716.0, 'train_loss': 1.3028948974609376, 'epoch': 3.6363636363636362})

In [37]:
trainer.evaluate()

  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 3.417602777481079,
 'eval_accuracy': 0.3181818127632141,
 'eval_runtime': 0.5748,
 'eval_samples_per_second': 38.273,
 'eval_steps_per_second': 38.273,
 'epoch': 3.6363636363636362}

In [38]:
trainer.push_to_hub(new_model_name)

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/loha_fine_tuned_cb/commit/966e6dc2d8279fbb73bbc20be7e2fd51d44e595b', commit_message='loha_fine_tuned_cb', commit_description='', oid='966e6dc2d8279fbb73bbc20be7e2fd51d44e595b', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [40]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
# set_seed(42)
import torch

adapter_name = "lenatr99/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [44]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
outputs = model(**inputs, labels=label)
logits = outputs.logits

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
# Print prediction
logits.argmax().item()

1